# **Housing Price Analysis in Poznań**

## Notebook 2: Data Processing and Transformation

Welcome to the second notebook of the *Housing Price Analysis in Poznań* project.  
In this notebook, we focus on processing the raw HTML data collected in the previous step and transforming it into a structured dataset.

### Objectives of this Notebook:
1. Parse HTML files containing housing data using **BeautifulSoup**.
2. Extract key attributes such as city, district, street, price, area, and other relevant details.
3. Clean and format the extracted data for consistency and usability.
4. Save the processed data in **JSON format** for subsequent analysis.

---

This notebook builds on the raw data collected in **Notebook 1** and prepares it for detailed exploration and visualization in the next steps of the project.


In [187]:
import json
from bs4 import BeautifulSoup

# Path to the directory
scrapping_date = "19-12-2024"
folder_path = f"../data/raw/{scrapping_date}"

# Range of page numbers to process
start_page = 1
end_page = 40

# List to store results
results = []

# Iterating through page numbers
for page_number in range(start_page, end_page + 1):
    # Generating file name
    file_name = f"output_page_{page_number}.html"
    file_path = f"{folder_path}/{file_name}"

    try:
        # Open and read the HTML file
        with open(file_path, "r", encoding="utf-8") as file:
            content = file.read()

        # Creating a BeautifulSoup object
        soup = BeautifulSoup(content, "html.parser")

        # Searching for values based on the specified tag structure
        cities = soup.select("div.card__outer > a:nth-child(2) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div > span > span:nth-child(1)")
        districts = soup.select("div.card__outer > a:nth-child(2) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div > span > span:nth-child(2)")
        streets = soup.select("div.card__outer > a:nth-child(2) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > span")
        prices_per_m = soup.select("div.card__outer > a:nth-child(2) > div:nth-child(2) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1)")
        prices = soup.select("div.card__outer > a:nth-child(2) > div:nth-child(2) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1)")
        areas = soup.select("div.property-info > span:nth-child(1)")
        rooms_number = soup.select("div.property-info")
        floors_number = soup.select("div.property-info")
        starts_date = soup.select("div.card__outer > a:nth-child(2) > div:nth-child(2) > div:nth-child(3) > div:nth-child(1) > div:nth-child(1)")
        links = soup.select("div.card__outer > a:nth-child(1)")
        
        listings_url = [link['href'] for link in links]
       

        # Iterating through cities and districts
        for city, district, street, price_per_m, price, area, room_number, floor_number, start_date, listing_url in zip(
            cities, districts, streets, prices_per_m, prices, areas, rooms_number, floors_number, starts_date, listings_url
        ):
            city_name = city.text.strip().rstrip("  •")
            district_name = district.text.strip().rstrip("  •")
            street_name = street.text.strip()
            price_per_m_name = price_per_m.text.strip()
            price_name = price.text.strip()
            area_name = area.text.strip()
            room_number_parts = room_number.text.strip().rstrip().split("•")
            floor_number_parts = floor_number.text.strip().rstrip().split("•")
            
            room_number_name = room_number_parts[1] if len(room_number_parts) > 1 else "Nieznane"
            floor_number_name = floor_number_parts[2] if len(floor_number_parts) > 2 else "Nieznane"

            start_date_name = start_date.text.strip()
            

            # Adding data to the results
            results.append({
                "City": city_name,
                "District": district_name,
                "Street": street_name,
                "Price": price_name,
                "Price_per_m": price_per_m_name,
                "Area": area_name,
                "Room number": room_number_name,
                "Floor number": floor_number_name,
                "Start date": start_date_name,
                "Scraping date": scrapping_date,
                "Listing url": listing_url
            })

    except FileNotFoundError:
        print(f"Plik {file_name} nie został znaleziony. Pomijam.")

# Saving results to a JSON file
output_file = f"../data/interim/results_{scrapping_date}.json"
with open(output_file, "w", encoding="utf-8") as json_file:
    json.dump(results, json_file, ensure_ascii=False, indent=4)

# Displaying results
for index, result in enumerate(results, start=1):
    print(f"Wynik {index}: Miasto: {result['City']}, Dzielnica: {result['District']}, Ulica: {result['Street']}, Cena: {result['Price']}, Cena za m2: {result['Price_per_m']}, Powierzchnia: {result['Area']}, Pokoi: {result['Room number']}, Piętro: {result['Floor number']}, Start date: {result['Start date']}, Scraping date: {result['Scraping date']}, Listing url: {result['Listing url']}")


Wynik 1: Miasto: Poznań  , Dzielnica: Jeżyce  , Ulica: Druskienicka, Cena: 598 000 zł, Cena za m2: 11 500 zł/m², Powierzchnia: 52 m², Pokoi: 2 pokoje, Piętro: piętro 4/4, Start date: Dodane 2024.08.28, Scraping date: 19-12-2024, Listing url: /nieruchomosci/mieszkanie-poznan-jezyce-druskienicka/ob/36721401
Wynik 2: Miasto: Poznań  , Dzielnica: Wilda  , Ulica: Dolna Wilda, Cena: 725 000 zł, Cena za m2: 11 885 zł/m², Powierzchnia: 61 m², Pokoi: 3 pokoje, Piętro: piętro 3/4, Start date: Dodane 2024.07.22, Scraping date: 19-12-2024, Listing url: /nieruchomosci/mieszkanie-poznan-wilda-dolna-wilda/ob/35271251
Wynik 3: Miasto: wielkopolskie  , Dzielnica: Poznań  , Ulica: Poznań, Cena: 619 000 zł, Cena za m2: 12 137 zł/m², Powierzchnia: 51 m², Pokoi: 2 pokoje, Piętro: piętro 7/7, Start date: Dodane 2024.06.09, Scraping date: 19-12-2024, Listing url: /nieruchomosci/mieszkanie-poznan-wilkonskich/oi/34832813
Wynik 4: Miasto: wielkopolskie  , Dzielnica: Poznań  , Ulica: Os. Rusa, Cena: 665 000 zł, 

In [188]:
import pandas as pd
from pprint import pprint

scrapping_date = "19-12-2024"
# Path to the JSON file
json_file_path = f"../data/interim/results_{scrapping_date}.json"

# Open and load JSON data
df = pd.read_json(json_file_path)
# Display data in a readable format
pprint(df)

                 City        District        Street       Price   Price_per_m  \
0            Poznań          Jeżyce    Druskienicka  598 000 zł  11 500 zł/m²   
1            Poznań           Wilda     Dolna Wilda  725 000 zł  11 885 zł/m²   
2     wielkopolskie          Poznań          Poznań  619 000 zł  12 137 zł/m²   
3     wielkopolskie          Poznań        Os. Rusa  665 000 zł   8 439 zł/m²   
4            Poznań    Stare Miasto          Grobla  270 000 zł  10 727 zł/m²   
...               ...             ...           ...         ...           ...   
1395         Poznań      Świerczewo      Świerczewo  540 000 zł   5 745 zł/m²   
1396         Poznań        Grunwald           Górki  450 000 zł  10 514 zł/m²   
1397         Poznań        Grunwald           Górki  498 000 zł  10 205 zł/m²   
1398         Poznań        Chartowo        Chartowo   40 000 zł   3 077 zł/m²   
1399  wielkopolskie          Poznań          Poznań  368 500 zł   6 888 zł/m²   

       Area Room number  Fl

In [189]:
# Data cleaning
df['Area'] = df['Area'].str.replace(' m²', '', regex=False)
df['Room number'] = df['Room number'].str.replace(' pokoje', '', regex=False)
df['Room number'] = df['Room number'].str.replace(' pokój', '', regex=False)
df['Room number'] = df['Room number'].str.replace(' pokoi', '', regex=False)
df['Floor number'] = df['Floor number'].str.replace('piętro ', '', regex=False)
df['Floor number'] = df['Floor number'].str.replace('parter/', '0/', regex=False)
df['Floor number'] = df['Floor number'].str.replace('parter', '0/0', regex=False)
df['Start date'] = df['Start date'].str.replace('Dodane ', '', regex=False)
df['Price'] = df['Price'].str.replace(' zł', '', regex=False)
df['Price_per_m'] = df['Price_per_m'].str.replace(' zł/m²', '', regex=False)
df['City'] = df['City'].str.replace('wielkopolskie', 'Poznań', regex=False)
df['District'] = df['District'].str.strip()
df["Listing url"] = "https://gratka.pl" + df["Listing url"]

# Splitting the "Floor Number" column into two columns: "Floor" and "Floors Number"
df[['Floor', 'Floors number']] = df['Floor number'].str.split('/', expand=True)
df = df.drop(columns='Floor number')

# Converting the column to datetime type
df['Start date'] = pd.to_datetime(df['Start date'], format='%Y.%m.%d')
df['Scraping date'] = pd.to_datetime(df['Scraping date'], format='%d-%m-%Y')

# Changing the date format to "dd.mm.yyyy"
df['Start date'] = df['Start date'].dt.strftime('%d.%m.%Y')
df['Scraping date'] = df['Scraping date'].dt.strftime('%d.%m.%Y')

# Removing rows containing "Nieznane"
df = df[~df.isin(['Nieznane']).any(axis=1)]

# Removing duplicates
df_cleaned = df.drop_duplicates()

# I assume that if the value of the entire building is not given, the given floor of the apartment is the last one
df_cleaned["Floors number"] = df_cleaned["Floors number"].fillna(df_cleaned["Floor"])

pprint(df_cleaned)

          City      District        Street    Price Price_per_m Area  \
0     Poznań          Jeżyce  Druskienicka  598 000      11 500   52   
1     Poznań           Wilda   Dolna Wilda  725 000      11 885   61   
2     Poznań          Poznań        Poznań  619 000      12 137   51   
3     Poznań          Poznań      Os. Rusa  665 000       8 439   78   
4     Poznań    Stare Miasto        Grobla  270 000      10 727   25   
...        ...           ...           ...      ...         ...  ...   
1310  Poznań          Poznań        Poznań  580 000      13 013   44   
1311  Poznań      Świerczewo    Świerczewo  540 000       5 745   94   
1312  Poznań        Grunwald         Górki  450 000      10 514   42   
1313  Poznań        Grunwald         Górki  498 000      10 205   48   
1314  Poznań        Chartowo      Chartowo   40 000       3 077   13   

     Room number  Start date Scraping date  \
0              2  28.08.2024    19.12.2024   
1              3  22.07.2024    19.12.2024 

/var/folders/y2/3xyrx43j34z8xvfcrh70w5280000gp/T/ipykernel_25319/1604573254.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned["Floors number"] = df_cleaned["Floors number"].fillna(df_cleaned["Floor"])


In [168]:
print(df_cleaned.dtypes)

City             object
District         object
Street           object
Price            object
Price_per_m      object
Area             object
Room number      object
Start date       object
Scraping date    object
Listing url      object
Floor            object
Floors number    object
dtype: object


In [186]:
unique_value = df_cleaned['Floors number'].unique()
print(unique_value)

['4' '7' '16' '5' '8' '0' '3' '1' '2' '6' '9' '10' '11' None '12' '15'
 '13' '17' '18' '14']


In [190]:
# Wczytanie pliku z ulicami i dzielnicami zeby dopasowac brakujace
df_streets_and_districts = pd.read_csv("../data/interim/streets_and_districts.csv")
df_streets_and_districts.head()

# Filtrujemy wiersze w df_cleaned, gdzie kolumna "District" ma wartość "Poznań"
poznan_rows = df_cleaned["District"] == "Poznań"

# Tworzymy słownik mapowania ulic do dzielnic na podstawie df_streets_and_districts
street_to_district = df_streets_and_districts.set_index("Street")["District"].to_dict()

# Aktualizujemy kolumnę "District" w df_cleaned dla wierszy z "Poznań"
# W przypadku braku dopasowania, wstawiamy "Nieznana"
df_cleaned.loc[poznan_rows, "District"] = df_cleaned.loc[poznan_rows, "Street"].map(street_to_district).fillna("Nieznana")

# Wyświetlamy wynik
print(df_cleaned)

          City      District        Street    Price Price_per_m Area  \
0     Poznań          Jeżyce  Druskienicka  598 000      11 500   52   
1     Poznań           Wilda   Dolna Wilda  725 000      11 885   61   
2     Poznań        Nieznana        Poznań  619 000      12 137   51   
3     Poznań     Nowe Miasto      Os. Rusa  665 000       8 439   78   
4     Poznań    Stare Miasto        Grobla  270 000      10 727   25   
...        ...           ...           ...      ...         ...  ...   
1310  Poznań        Nieznana        Poznań  580 000      13 013   44   
1311  Poznań      Świerczewo    Świerczewo  540 000       5 745   94   
1312  Poznań        Grunwald         Górki  450 000      10 514   42   
1313  Poznań        Grunwald         Górki  498 000      10 205   48   
1314  Poznań        Chartowo      Chartowo   40 000       3 077   13   

     Room number  Start date Scraping date  \
0              2  28.08.2024    19.12.2024   
1              3  22.07.2024    19.12.2024 

In [191]:
# Saving the DataFrame to a CSV file
df_cleaned.to_csv('output.csv', index=False)

# Displaying a success message
print("DataFrame zapisano do pliku 'output.csv'")

DataFrame zapisano do pliku 'output.csv'
